In [4]:
from tabulate import tabulate
import torch

判断link里面预测为正的结果，且rel里面预测和ground truth一致的标签

In [7]:
def judge_link_rel(link_dict, rel_dict, model_name=None):
    # 处理link预测的结果
    indexes = torch.where(link_dict['true_label'] == 1)  # 真实存在的link：在true label里面是1的边
    links_pred_edge = link_dict['pred_edge_index'].t()[indexes].tolist()
    links_true_label = link_dict['true_label'][indexes].tolist()
    links_pred_label = link_dict['pred_label'][indexes].squeeze().tolist()

    assert len(links_pred_edge) == len(links_true_label) == len(links_pred_label)  # 断言预测的边和真实标签和预测的结果长度一样
    links_pred_dict = {tuple(e):[int(tl), pl] for e, tl, pl in zip(links_pred_edge, links_true_label, links_pred_label)}  # 每个边对应的真实标签和预测标签生成字典

    # 处理rel预测的结果
    rels_edge_index = rel_dict['pred_edge_index'].T.tolist()
    rels_true_label = rel_dict['true_label'].tolist()
    rels_pred_label = rel_dict['pred_label'].squeeze().tolist()

    assert len(rels_edge_index) == len(rels_true_label) == len(rels_pred_label)
    rels_pred_dict = {tuple(e):[int(tl), pl] for e, tl, pl in zip(rels_edge_index, rels_true_label, rels_pred_label)}  # 每个边对应的在关系中的真实标签和预测标签

    count = 0
    for k, v in rels_pred_dict.items():
        link_result = links_pred_dict[k]
        if v[0] == v[1] and link_result[0] == link_result[1]:
            count += 1

    return count / len(rels_pred_dict)

In [8]:
import torch

link_file_name = '/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_GraphSage_link_0_2019.pth'
rel_file_name = '/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_GraphSage_rel0_2019.pth'

link_result_dict = torch.load(link_file_name)
rel_result_dict = torch.load(rel_file_name)

print(judge_link_rel(link_result_dict, rel_result_dict))

0.8662218749373886


In [9]:
from tabulate import tabulate
table_header, table_data = ['neo', 'GraphSage', 'GCN', 'GAT', 'Random-feature', 'bgp2vec'], []
   
for i in range(20):
    ms = []
    for m in table_header:
        link_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_{m}_link_{i}_2024.pth'
        rel_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_{m}_rel{i}_2024.pth'
        
        if m == 'neo':
            link_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_logs/0118_result_logs/link_results/best_performance_{m}_link_{i}_2024.pth'
            rel_file_name = f'/data/taozekun/workspace/ed_rel/datasets/data_augmentation/neo_save/best_performance_{m}_rel_{i}_2024.pth'
        
        elif m == 'Random-feature':
            m = 'neo'
            link_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/random_feature_restults_stores/best_performance_{m}_link_{i}_2024.pth'
            rel_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/random_feature_restults_stores/best_performance_{m}_rel{i}_2024.pth'
        
        elif m == 'bgp2vec':
            link_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_GCN_link_{i}_2024.pth'
            rel_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_{m}_rel{i}_2024.pth'
            gcn_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_GCN_rel{i}_2024.pth'
            
        
        link_result_dict = torch.load(link_file_name)
        rel_result_dict = torch.load(rel_file_name)
        if m == 'bgp2vec':
            rel_result_dict['pred_edge_index'] = torch.load(gcn_file_name)['pred_edge_index']
    
        acc = judge_link_rel(link_result_dict, rel_result_dict)
        ms.append(acc)
    table_data.append(ms)

print(tabulate(table_data, headers=table_header, tablefmt='simple'))

     neo    GraphSage       GCN       GAT    Random-feature    bgp2vec
--------  -----------  --------  --------  ----------------  ---------
0.972116     0.918769  0.954515  0.965765          0.968416   0.950342
0.968815     0.913719  0.96641   0.969226          0.970422   0.964157
0.968571     0.896583  0.962298  0.943965          0.961367   0.958913
0.973358     0.916519  0.973979  0.955197          0.972435   0.9686
0.974002     0.926395  0.964981  0.938928          0.971884   0.961689
0.966274     0.893782  0.947463  0.937519          0.968148   0.943887
0.972148     0.879793  0.967863  0.964338          0.973181   0.965954
0.970797     0.906613  0.954424  0.930709          0.968778   0.940236
0.979046     0.920983  0.961663  0.95074           0.973975   0.94826
0.959706     0.886726  0.941302  0.937735          0.960208   0.92424
0.969657     0.906589  0.946499  0.953182          0.961003   0.944263
0.973801     0.914436  0.957706  0.941716          0.968397   0.95444
0.963043   

In [8]:
from tabulate import tabulate
table_header, table_data = ['neo', 'GraphSage', 'GCN', 'GAT'], []
   
for i in range(20):
    ms = []
    for m in table_header:
        link_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_{m}_link_{i}_2019.pth'
        rel_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_{m}_rel{i}_2019.pth'
        
        if m == 'neo':
            link_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_logs/0118_result_logs/link_results/best_performance_{m}_link_{i}_2019.pth'
            rel_file_name = f'/data/taozekun/workspace/ed_rel/datasets/data_augmentation/neo_save/best_performance_{m}_rel_{i}_2019.pth'
        
        elif m == 'Random-feature':
            m = 'neo'
            link_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/random_feature_restults_stores/best_performance_{m}_link_{i}_2019.pth'
            rel_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/random_feature_restults_stores/best_performance_{m}_rel{i}_2019.pth'
        
        elif m == 'bgp2vec':
            link_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_GCN_link_{i}_2019.pth'
            rel_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_{m}_rel{i}_2019.pth'
            gcn_file_name = f'/data/taozekun/workspace/ed_rel/models/experiment_result_stores/best_performance_GCN_rel{i}_2019.pth'
            
        
        link_result_dict = torch.load(link_file_name)
        rel_result_dict = torch.load(rel_file_name)
        if m == 'bgp2vec':
            rel_result_dict['pred_edge_index'] = torch.load(gcn_file_name)['pred_edge_index']
    
        acc = judge_link_rel(link_result_dict, rel_result_dict)
        ms.append(acc)
    table_data.append(ms)

print(tabulate(table_data, headers=table_header, tablefmt='simple'))

     neo    GraphSage       GCN       GAT
--------  -----------  --------  --------
0.975797     0.866222  0.928473  0.88758
0.981674     0.893197  0.931368  0.917284
0.975806     0.883746  0.920866  0.891957
0.979149     0.888391  0.931929  0.886648
0.979637     0.941984  0.970247  0.923279
0.9825       0.910549  0.94245   0.911209
0.980669     0.885721  0.926803  0.912164
0.978591     0.897781  0.930274  0.914854
0.977503     0.862773  0.920862  0.896768
0.976327     0.869384  0.932856  0.892408
0.974852     0.896193  0.924912  0.916836
0.98583      0.929586  0.968409  0.947226
0.977392     0.921973  0.956001  0.926793
0.984734     0.912696  0.949911  0.911967
0.978347     0.92562   0.963896  0.928458
0.981624     0.907104  0.958269  0.968249
0.984463     0.901714  0.942737  0.938378
0.982146     0.874937  0.923497  0.941942
0.978664     0.933541  0.963549  0.965406
0.982487     0.905596  0.969525  0.930073


### Log 处理

In [8]:
import os
from tabulate import tabulate

In [9]:
log_file = "/data/taozekun/workspace/ed_rel/models/experiment_result_logs/random_feature_logs/experiment_link_random_feature_neo_2024.log"
os.path.exists(log_file)

True

In [12]:
table_header, table_data = ['auc', 'accuracy', 'precision', 'recall', 'f1'], []
with open(log_file, 'r') as f:
    for line in f:
        if line.startswith('auc'):
            metric_list = line.strip().split(', ')
            metrics = [float(m.split(':')[1]) for m in metric_list]
            table_data.append(metrics)

print(tabulate(table_data, headers=table_header, tablefmt='simple'))

auc    accuracy    precision    recall    f1
-----  ----------  -----------  --------  ----


In [3]:
os.path.exists(log_file)

True

In [13]:
rel_log_file = "/data/taozekun/workspace/ed_rel/models/experiment_result_logs/random_feature_logs/experiment_rel_random_feature_neo_2024.log"
os.path.exists(log_file)

True

In [24]:

table_header, table_data = ['auc', 'accuracy', 'precision', 'recall', 'f1', 'Pos_F1'], []
with open(rel_log_file, 'r') as f:
    for line in f:
        if line.startswith('AUC'):
            metric_list = line.strip().split(', ')
            metrics = [float(m.split(':')[1].strip()) for m in metric_list]
            table_data.append(metrics)

print(tabulate(table_data, headers=table_header, tablefmt='simple'))

    auc    accuracy    precision    recall       f1    Pos_F1
-------  ----------  -----------  --------  -------  --------
86.0262     98.226       69.0871   34.924   99.0981   46.395
83.5522     98.307       57.3427   26.4858  99.1421   36.2351
90.9975     97.9879      59.6735   44.4194  98.9729   50.9288
93.5634     98.7964      69.8768   47.7795  99.3897   56.7531
84.4454     97.9719      68.1628   29.7359  98.9681   41.4077
87.9922     97.5526      63.875    24.7698  98.7526   35.6968
88.0027     98.3201      66.6667   34.2593  99.147    45.2599
90.8487     97.8304      74.5763   52.3207  98.8838   61.4968
92.7427     97.9703      62.7768   38.0485  98.9652   47.3803
93.8304     97.8639      74.7214   46.9584  98.9043   57.6727
81.2646     97.098        0         0       98.5276  nan
87.5579     97.5434      67.3767   25.0312  98.7475   36.5017
91.7902     97.0639      69.7044   37.1879  98.4889   48.5004
84.8871     98.7032      66.7219   27.5838  99.3446   39.0315
82.0776     97